In [ ]:
import itertools
from opencage.geocoder import OpenCageGeocode
from bs4 import BeautifulSoup as bs
import numpy as np
from selenium import webdriver
from splinter import Browser
import pandas as pd
import os
import time
import requests
import warnings
warnings.filterwarnings('ignore')

In [ ]:
path = "Aviation_Accident_Report_df2.csv"
aar_csv = pd.read_csv(path)
aar_csv

In [ ]:
cities = aar_csv["City"].to_list()
states = aar_csv["State"].to_list()

key = "8006a222e9ce4db7a69d9e8ba04f10e8"

lati = []
longi = []


for (city, state) in zip(cities, states):
    try:
        geocoder = OpenCageGeocode(key)
        query = f'{city}, {state}'
        results = geocoder.geocode(query)
        lat = results[0]['geometry']['lat']
        lng = results[0]['geometry']['lng']
        lati.append(lat)
        longi.append(lng)
        time.sleep(1)
    except KeyError:
        print(city)


In [ ]:

aar_csv['Latitude'] = lati
aar_csv['Longitude'] = longi

In [ ]:
aar_csv
aar_csv.to_csv('Aviation_Accident_Report_df2.csv', index = False)


In [ ]:
#Just testing stuff.... this is probably useless so ignore it please. Or don't. It's up to you really.

url ='https://aviation-safety.net/statistics/worst/worst.php?list=worstground'
#soup = BeautifulSoup(requests.get(the_url).text)
#aar_tables = pd.read_html(str(soup.find_all('table')[9]))
r = requests.get(url)
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(r)

In [ ]:


driver = webdriver.Chrome()
driver.get("https://aviation-safety.net/statistics/worst/worst.php?list=worstground")

html = driver.page_source

tables = pd.read_html(html)

driver.close()



In [ ]:
# THE DEED IS DONE AND I GOT THE TABLE

deaths = tables[0]
deaths

In [ ]:

#Url params
pages = [1]
years = np.arange().tolist()

#List results
Fatalities = []
Dates = []
Type = []
Registration = []
Operator = []
Location = []
Cat = []


for year in years:
    for page in pages:
        try:
            driver = webdriver.Chrome()
            driver.get(f"https://aviation-safety.net/statistics/worst/worst.php?list=worstground")
            html = driver.page_source
            a_tables = pd.read_html(html)
            accidents = a_tables[0]
            Fatalities.append(accidents["fat."].to_list())
            Registration.append(accidents["registration"].to_list())
            Dates.append(accidents["date"].to_list())
            Type.append(accidents["type"].to_list())
            Operator.append(accidents["operator"].to_list())
            Location.append(accidents["location"].to_list())
            Cat.append(accidents["cat"].to_list())
            
            driver.close()
        
        except (KeyError, ValueError):
            print("Page not found")
            
            driver.close()
        
        





In [ ]:

new_Dates = list(itertools.chain.from_iterable(Dates))
new_Type = list(itertools.chain.from_iterable(Type))
new_Registration = list(itertools.chain.from_iterable(Registration))
new_Operator = list(itertools.chain.from_iterable(Dates))
new_Fatalities = list(itertools.chain.from_iterable(Fatalities))
new_Location = list(itertools.chain.from_iterable(Location))
new_Cat = list(itertools.chain.from_iterable(Cat))


In [ ]:
mistakes = {'Date': new_Dates,
        'Type': new_Type,
        'Registration': new_Registration,
        'Operator': new_Operator,
        'Fatalities': new_Fatalities,
        'Location': new_Location,
        'Cat': new_Cat
        }

Accident_df = pd.DataFrame(mistakes, columns = ['Date', 'Type', 'Registration',
                                               'Operator', 'Fatalities', 'Location', 'Cat'])

Accident_df

In [ ]:
Accident_df.to_csv('worst_accident.csv', index = False)